# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego
Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego. 

### Autor rozwiązania
Uzupełnij poniższe informacje umieszczając swoje imię i nazwisko oraz numer indeksu:

In [14]:
NAME = "Bartłomiej Andree"
ID = "162961"

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. **Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.**

Schemat oceniania:
- wynik na zbiorze Iris (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 10%: +40%,
- wynik na ukrytym zbiorze testowym 1 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 15%: +30%,
- wynik na ukrytym zbiorze testowym 2 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 5%: +30%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn). 
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.

In [15]:
import numpy as np

class TreeNode:
    def __init__(self):
        self.left: TreeNode | None = None   # wierzchołek znajdujący się po lewej stronie
        self.right: TreeNode | None = None  # wierzchołek znajdujący się po prawej stronie
        self.value = None                   # wartość liścia (przypisana odpowiedź)
        self.split_feature = None           # indeks cechy, po której dzielimy (tutaj minimalnie tylko 0)
        self.split_threshold = None         # próg podziału

    def fit(self, data: np.ndarray, target: np.ndarray) -> None:
        """
        Args:
            data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
            target (np.ndarray): wektor klas o długości n, gdzie n to liczba przykładów
        """

        # Poniej znajdziesz przykładowy "pseudo-kod" rozwiązania, nie musisz się go trzymać
		# (możesz zaimplementować to w inny sposób, jeżeli wolisz)
		#
		# Znajdź najlepszy podział x, y
		# if uzyskano poprawę funkcji celu (bądź inny, zaproponowany przez Ciebie warunek):
		# 	podziel dane na dwie części data_left i data_right, zgodnie z warunkiem
		# 	self.left = Node()
		# 	self.right = Node()
		# 	self.left.fit(data_left)
		# 	self.right.fit(data_right)
		# else:
		# 	obecny Node jest liściem, zapisz jego odpowiedź


        # Jeśli wszystkie etykiety są takie same, ustawiamy liść i zapisujemy jego odpowiedź
        if len(np.unique(target)) == 1:
            self.value = target[0]
        else:
            # W tym minimalnym przykładzie jako kryterium podziału wybieramy medianę pierwszej cechy
            threshold = np.median(data[:, 0])

            # Dzielimy dane na dwie części: data_left oraz data_right, zgodnie z warunkiem
            left_indices = data[:, 0] <= threshold
            right_indices = data[:, 0] > threshold

            # Jeśli podział nie rozdziela danych, ustawiamy węzeł jako liść (głosowanie większościowe)
            if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                self.value = np.bincount(target).argmax()
                return

            # Zapisujemy warunek podziału w bieżącym węźle
            self.split_feature = 0
            self.split_threshold = threshold

            # Przygotowanie danych dla lewego i prawego podziału
            data_left = data[left_indices]
            target_left = target[left_indices]
            data_right = data[right_indices]
            target_right = target[right_indices]

            # Utwórz węzły potomne i rekurencyjnie dopasuj poddrzewa
            self.left = TreeNode()
            self.right = TreeNode()
            self.left.fit(data_left, target_left)
            self.right.fit(data_right, target_right)

    def predict(self, data: np.ndarray) -> np.ndarray:
        """
        Args:
            data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech

        Returns:
            np.ndarray: wektor przewidzianych klas o długości n, gdzie n to liczba przykładów
        """
        y_pred = np.zeros(data.shape[0])

        # Poniżej znajdziesz przykładowy "pseudo-kod" rozwiązania, nie musisz się go trzymać
		# (możesz zaimplementować to w inny sposób, jeżeli wolisz),
		# ważne by metoda TreeNode.predict zwracała wektor przewidzianych klas
		#
        # Dla każdego przykładu w data:
		#   node = self
        #   if node nie jest liściem:
        #       if warunek podziału node jest spełniony:
        #           node = node.right
        #       else:
        #           node = node.left
        #   y_pred[i] = zwróć wartość node (liść)

        # Dla każdego przykładu przechodzimy po drzewie aż do liścia
        for i, sample in enumerate(data):
            node = self
            while node.value is None:
                # Jeśli warunek podziału jest spełniony, przechodzimy do prawego poddrzewa,
                # w przeciwnym razie do lewego
                if sample[node.split_feature] > node.split_threshold:
                    node = node.right
                else:
                    node = node.left
            y_pred[i] = node.value
        return y_pred


## Przykład trenowanie i testowania drzewa
 
Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).

In [16]:
#!pip install scikit-learn

In [17]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.6
